# Part 3: Horse Race Prediction
## Classification Modelling
- In this section, we want to classify if a horse can win the race, and which three horses will be ranked in the top 3.

In [ ]:
!pip install imblearn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
%load_ext autotime

import warnings
warnings.filterwarnings("ignore")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, GridSearchCV

from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from sklearn.metrics import f1_score, roc_curve, auc, confusion_matrix, average_precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

import joblib
import pickle
import time

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [ ]:
# Read the train and test files
df_train1 = pd.read_csv('D:\\documentos\\IA Caballos\\hurdle_data\\df_train.csv', index_col=0)
df_test1 = pd.read_csv('D:\\documentos\\IA Caballos\\hurdle_data\\df_test.csv', index_col=0)
df_unseen1 = pd.read_csv('D:\\documentos\\IA Caballos\\hurdle_data\\df_unseen.csv', index_col=0)

In [ ]:
#df_train = df_train1.dropna()
df_train = df_train1
df_train

In [ ]:
#df_test = df_test1.dropna()
df_test = df_test1
df_test

In [ ]:
#df_unseen = df_unseen1.dropna()
df_unseen = df_unseen1
df_unseen

In [ ]:
df_train.fillna(0, inplace=True)
df_train

In [ ]:
df_test1.fillna(0, inplace=True)
df_test1

In [ ]:
df_unseen.fillna(0, inplace=True)
df_unseen

In [ ]:
# View the shape of the train and test files
print(df_train.shape)
print(df_test1.shape)
print(df_unseen.shape)

In [ ]:
# View the first 2 rows of the train file
df_train.head(2)

In [ ]:
# View the first 2 rows of the test files
df_test1.head(2)

In [ ]:
# View the first 2 rows of the unseen file
df_unseen.head(2)

In [ ]:
import re
from numba import jit, cuda

@jit(target_backend='cuda')
def convert_to_km(distance):
    '''
    distance can be a string with km or m as units
    e.g. 300km, 1.1km, 200m, 4.5m
    '''
    
    # split the string into value and unit ['300', 'km']
    #split_dist = re.match('([\d\.]+)?([a-zA-Z]+)', distance)
    split_dist = re.findall('([\d\.]+)?([a-zA-Z]+)', distance, re.U)
    print(split_dist)
    dist = 0.0
    dist1 = 0.0
    dist2 = 0.0
    dist3 = 0.0
    for value in split_dist:
        if value[1] == 'm':
            dist1 = float(value[0])
        elif value[1] == 'f':
            dist2 = float(value[0])*0.125
        elif value[1] == 'y':
            dist3 = float(value[0])*0.0005681818
        else:
            pass
      
    dist = dist1 + dist2 + dist3
    
    return dist

In [ ]:
df_train.info()

In [ ]:
df_train['race_distance'] = df_train.apply(lambda row: convert_to_km(row['race_distance']), axis=1)

df_train.head(2)

In [ ]:
df_test['race_distance'] = df_test.apply(lambda row: convert_to_km(row['race_distance']), axis=1)

df_test.head(2)

In [ ]:
df_unseen['race_distance'] = df_unseen.apply(lambda row: convert_to_km(row['race_distance']), axis=1)

df_unseen.head(2)

In [ ]:
df_train


In [ ]:
df_test1


In [ ]:
df_unseen

In [ ]:
df_test1['win_odds'] = df_test1['win_odds'].str.rstrip('%').astype('float') / 100.0
df_train['win_odds'] = df_train['win_odds'].str.rstrip('%').astype('float') / 100.0
df_unseen['win_odds'] =df_unseen['win_odds'].str.rstrip('%').astype('float') / 100.0
#jockey_percentage_wins
#df_test['jockey_percentage_wins'] = df_test['jockey_percentage_wins'].str.rstrip('%').astype('float') / 100.0
#df_train['jockey_percentage_wins'] = df_train['jockey_percentage_wins'].str.rstrip('%').astype('float') / 100.0
#df_unseen['jockey_percentage_wins'] =df_unseen['jockey_percentage_wins'].str.rstrip('%').astype('float') / 100.0

In [ ]:
# Keep the features we want to train our model on
X_train = df_train[[#'actual_weight', 
                    'declared_horse_weight',
                    'draw', 'win_odds', 
                    'jockey_ave_rank',
                    'trainer_ave_rank', 
                    'recent_ave_rank',
                    'race_distance', 
                     'jockey_hurdle_rate','trainers_hurdle_rate']]

y_train = df_train[['HorseWin', 'HorseRankTop3', 'HorseRankTop50Percent']]

# Keep the features we want to train our model on
X_test = df_test1[[#'actual_weight',
                  'declared_horse_weight',
                   'draw', 'win_odds',
                   'jockey_ave_rank',
                    'trainer_ave_rank', 
                    'recent_ave_rank', 
                    'race_distance', 
                     'jockey_hurdle_rate','trainers_hurdle_rate']]
                    
y_test = df_test1[['HorseWin', 'HorseRankTop3', 'HorseRankTop50Percent']]

# Keep the features we want for the unseen data
X_unseen = df_unseen[[#'actual_weight', 
                      'declared_horse_weight',
                     'draw', 'win_odds',
                        'jockey_ave_rank',
                        'trainer_ave_rank', 'recent_ave_rank', 
                        'race_distance', 
                     'jockey_hurdle_rate','trainers_hurdle_rate']]



In [ ]:
# View the shape of the train and test files
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(df_unseen.shape)

In [ ]:
# Check the mean of the target variable
y_train.mean()

We can see that there is data imbalance for the HorseWin and HorseRankTop3 variables, so we need to account for these later when modelling.

In [ ]:
# Specify kfold cross validation
kfold = KFold(n_splits=5)

### Define functions to run and evaluate models

In [ ]:
# Create an empty dataframe to store the results
df_results = pd.DataFrame(columns=['Model', 'Prediction', 'CV-F1', 'F1 Score', 'AUC', 'Recall', 'Precision'])

In [ ]:
# Define a function to run the model
def run_model(model, X_train, y_train, X_test, y_test, kfold):

    # Create a dataframe to store the predictions for the UNSEEN data
    df_pred = pd.DataFrame()
    df_pred['RaceID'] = df_unseen['race_id']
    df_pred['HorseID'] = df_unseen['horse_id']

    # Store model name
    model_name = str(model).split('(')[0]

    for column in ['HorseWin', 'HorseRankTop3']:

        # Print the column name
        print(f"Results for model {model_name} and target variable {column}:")
        
        # Fit the model
        model.fit(X_train, y_train[column].to_numpy())
        
        # Calculate the cross validation score
        cv_score = cross_val_score(model, X_train, y_train[column].to_numpy(), cv=kfold, scoring='f1_weighted').mean()
        cv_score = round(cv_score, 3)
        
        # Make predictions
        y_pred = model.predict(X_test)
        y_unseen_pred = model.predict(X_unseen)

        # Store the predictions in the dataframe
        df_pred[column] = y_unseen_pred
        
        # Calculate the f1 score
        f1 = f1_score(y_test[column].to_numpy(), y_pred, average='weighted')
        f1 = round(f1, 3)
        
        # Calculate PR AUC
        pr_auc = average_precision_score(y_test[column].to_numpy(), y_pred, average='weighted')
        pr_auc = round(pr_auc, 3)

        # Calculate Recall
        tn, fp, fn, tp = confusion_matrix(y_test[column].to_numpy(), y_pred).ravel()
        recall = tp / (tp + fn)
        recall = round(recall, 3)

        # Calculate Precision
        precision = tp / (tp + fp)
        precision = round(precision, 3)

        # Append the results to the dataframe
        df_results.loc[len(df_results)] = [model_name, column, cv_score, f1, pr_auc, recall, precision]

        # Print the results
        print('Cross Validation Score (F1-weighted): ', cv_score)
        print('F1 Score: ', f1)
        print('PR AUC (Avg Precision): ', pr_auc)
        print('Recall: ', recall)
        print('Precision: ', precision)

        # Plot confusion matrix
        plot_confusion_matrix(y_test, y_pred, column)

        # Plot precision recall curve
        plot_pr_auc(X_test, y_test, model, column)

    return df_pred

In [ ]:
# Define plot confusion matrix function
def plot_confusion_matrix(y_test, y_pred, column):
    # Calculate the confusion matrix
    cm = confusion_matrix(y_test[column].to_numpy(), y_pred)
    cm = pd.DataFrame(cm, columns=['Predicted 0', 'Predicted 1'], index=['Actual 0', 'Actual 1'])

    # Plot the confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix for ' + column)
    plt.show()

In [ ]:
# Define plot PR AUC function
def plot_pr_auc(X_test, y_test, model, column):
    # Get the probabilities of the predictions
    win_prob = model.predict_proba(X_test)[:, 1]

    # Get the precision and recall
    precision, recall, thresholds = precision_recall_curve(y_test[column], win_prob)

    # Calculate proportion of positive class
    proportion_pos_class = y_test[column].mean()
    
    # Print the PR AUC score
    pr_auc = round(auc(recall, precision),3)
    print(f'PR AUC score for {column}:', pr_auc)

    # Plot the PR AUC curve
    plt.figure(figsize = (8, 6))
    plt.plot([0, 1], [proportion_pos_class, proportion_pos_class], linestyle = '--')
    plt.plot(recall, precision, marker = '.')
    plt.title(f'PR AUC Curve for {column}')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.show()    

### Model 1: Logistic Regression

In [ ]:
# Initialize the model
lr = LogisticRegression()

In [ ]:
y_train.fillna(method = 'bfill')

In [ ]:
X_train.dropna(how='all')

In [ ]:
# Do hyperparameter tuning
param_grid = {'C': [0.0001, 0.001, 0.01, 0.1, 1, 10],
                'penalty': ['l1', 'l2'],
              'solver':['liblinear']}

grid = GridSearchCV(lr, param_grid, cv=kfold, scoring='f1_weighted', verbose=1, n_jobs=-1)

# find the best parameters
grid.fit(X_train, y_train['HorseWin'].to_numpy())

# Print the best parameters
print(grid.best_params_)
print(grid.best_score_)

# Initialize the model using best parameters
lr = grid.best_estimator_

In [ ]:
# Run Logistics Regression model
lr_pred = run_model(lr, X_train, y_train, X_test, y_test, kfold)

In [ ]:
# Print the results
df_results

### Model 2: Naive Bayes

In [ ]:
# Initialize the model
gnb = GaussianNB()

In [ ]:
# Do hyperparameter tuning
param_grid = {'var_smoothing': [1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]}

grid = GridSearchCV(gnb, param_grid, cv=kfold, scoring='f1_weighted', verbose=1, n_jobs=-1)

# find the best parameters
grid.fit(X_train, y_train['HorseWin'].to_numpy())

# Print the best parameters
print(grid.best_params_)
print(grid.best_score_)

# Initialize the model using best parameters
gnb = grid.best_estimator_

In [ ]:
# Run Naive Bayes model
gnb_pred = run_model(gnb, X_train, y_train, X_test, y_test, kfold)

In [ ]:
# Print the results
df_results

### Model 3: Random Forest Classifier

In [ ]:
# Initialize the model
rfc = RandomForestClassifier(n_estimators = 100, random_state = 42)

In [ ]:
# Do hyperparameter tuning
param_grid = {'max_depth': [20, 40],
                'max_features': [5, 10],
                'min_samples_leaf': [1, 2],
                'min_samples_split': [1, 2]}
grid = GridSearchCV(rfc, param_grid, cv=kfold, scoring='f1_weighted', verbose=1, n_jobs=-1)

# find the best parameters
grid.fit(X_train, y_train['HorseWin'].to_numpy())

# Print the best parameters
print(grid.best_params_)
print(grid.best_score_)

# Initialize the model using best parameters
rfc = grid.best_estimator_

In [ ]:
# Run Random Forest model
rfc_pred = run_model(rfc, X_train, y_train, X_test, y_test, kfold)

In [ ]:
# Print the results
df_results

### Model 4: SMOTE + Random Forest Classifier

In [ ]:
# Smote the training data
sm = SMOTE(random_state = 42)
rfc = RandomForestClassifier(max_depth=10, random_state = 42)

# Steps for the pipeline
steps = [('smote', sm), ('rfc', rfc)]

# Create the pipeline
smote_rfc = Pipeline(steps = steps)

In [ ]:
# Run the model
smote_rfc_pred = run_model(smote_rfc, X_train, y_train, X_test, y_test, kfold)

In [ ]:
# Print the results
df_results

### View the final results of our 4 classification models

In [ ]:
# View the results
df_results.sort_values(by='Prediction', ascending=False)

We will choose the Random Forest Classifer as our deployed model as it has good generalisation between the train and test F1 scores, and the highest PR AUC.

In [ ]:
# Save our models to csv
lr_pred.to_csv('D:\\documentos\\IA Caballos\\hurdle_data\\lr_pred.csv', index=False)
gnb_pred.to_csv('D:\\documentos\\IA Caballos\\hurdle_data\\gnb_pred.csv', index=False)
rfc_pred.to_csv('D:\\documentos\\IA Caballos\\hurdle_data\\rfc_pred.csv', index=False)
smote_rfc_pred.to_csv('D:\\documentos\\IA Caballos\\hurdle_data\\smote_rfc_pred.csv', index=False)

In [ ]:
# Save our Pipeline model to a file
joblib.dump(smote_rfc, 'D:\\documentos\\IA Caballos\\hurdle_data\\smote_rfc_model.pkl')

In [ ]:
# Save results to csv
df_results.to_csv('D:\\documentos\\IA Caballos\\hurdle_data\\class_results.csv', index=False)
